In [2]:
from config import access_token, user_id
import requests
from pprint import pprint
from datetime import date
import schedule
import time

In [3]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

/Users/gavinpayne/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/ipykernel_launcher.py:3: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Creates classes to represent schema of our database

class Steps(Base):
    __tablename__ = 'intraday_steps'
    id = Column(Integer, primary_key=True)
    date = Column(String)
    time = Column(String)
    steps = Column(Integer)
    
    
class Runs(Base):
    __tablename__ = 'Runs'
    id = Column(Integer, primary_key=True)
    date = Column(String)
    duration = Column(Integer)
    distance= Column(Float)
    pace = Column(Float)

class Heartrate(Base):
    __tablename__ = 'intraday_heartrate'
    id = Column(Integer, primary_key=True)
    date = Column(String)
    time = Column(String)
    rate = Column(Integer)
    

In [5]:
Base.metadata.tables

FacadeDict({'intraday_steps': Table('intraday_steps', MetaData(), Column('id', Integer(), table=<intraday_steps>, primary_key=True, nullable=False), Column('date', String(), table=<intraday_steps>), Column('time', String(), table=<intraday_steps>), Column('steps', Integer(), table=<intraday_steps>), schema=None), 'Runs': Table('Runs', MetaData(), Column('id', Integer(), table=<Runs>, primary_key=True, nullable=False), Column('date', String(), table=<Runs>), Column('duration', Integer(), table=<Runs>), Column('distance', Float(), table=<Runs>), Column('pace', Float(), table=<Runs>), schema=None), 'intraday_heartrate': Table('intraday_heartrate', MetaData(), Column('id', Integer(), table=<intraday_heartrate>, primary_key=True, nullable=False), Column('date', String(), table=<intraday_heartrate>), Column('time', String(), table=<intraday_heartrate>), Column('rate', Integer(), table=<intraday_heartrate>), schema=None)})

In [6]:
# Create our database engine
engine = create_engine('sqlite:///fitbit.sqlite')
conn = engine.connect()

In [7]:
# This is where we create our tables in the database
Base.metadata.create_all(conn)
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)


In [8]:
today = date.today()
today=today.strftime("%Y-%m-%d")

header = {'Authorization': 'Bearer '+access_token}
url = f'https://api.fitbit.com/1/user/{user_id}/'
steps_request = 'activities/steps/date/today/today/5min.json'
steps_response = requests.get(f'{url}{steps_request}',headers=header).json()

session = Session()

#STEPS INTRADAY INSTANCE#
for row in steps_response['activities-steps-intraday']['dataset']:
    session.add(Steps(date=today,time=row['time'],steps=row['value']))
session.commit()
session.close()
    



In [24]:
session.query(Steps.date,Steps.time,Steps.steps).all()
session.query(Steps).delete()
session.commit()

In [10]:
#FUNTION MAKE RUN REQUEST
today = date.today()
today=today.strftime("%Y-%m-%d")

header = {'Authorization': 'Bearer '+ access_token}
url = f'https://api.fitbit.com/1/user/{user_id}/'
request = f'activities/list.json?afterDate={today}&sort=asc&offset=0&limit=10'
run_request = requests.get(f'{url}{request}',headers=header).json()

session = Session()
#RUN SUMMARY INSTANCE#
for row in run_request['activities']:
    if row['activityName']=='Run':
        session.add(Runs(date=today,duration=row['activeDuration'],distance=row['distance'],pace=row['pace']))
session.commit()
session.close()     


In [25]:
session.query(Runs.date,Runs.duration,Runs.distance,Runs.pace).all()
session.query(Runs).delete()
session.commit()

In [12]:
#FUNCTION MAKE HEART RATE REQUEST
today = date.today()
today=today.strftime("%Y-%m-%d")

header = {'Authorization': 'Bearer '+ access_token}
url = f'https://api.fitbit.com/1/user/{user_id}/'
request = f'activities/heart/date/{today}/1d/5min.json'
heart_response=requests.get(f'{url}{request}',headers=header).json()

session = Session()
#HEARTRATE INTRADAY INSTANCE#
for row in heart_response['activities-heart-intraday']['dataset']:
    session.add(Heartrate(date=today,time=row['time'], rate=row['value']))

session.commit()
session.close()    



In [26]:
session.query(Heartrate.date,Heartrate.time,Heartrate.rate).all()
session.query(Heartrate).delete()
session.commit()